In [75]:
class Rewriter:
    
    def __init__(self):
        self.rules = []
    
    def add_rule(self, s, t):
        rule = (s, t)
        self.rules.append(rule)
    
    def apply(self, s):
        change = True
        while change:
            change = False
            for rule in self.rules:
                i = s.find(rule[0])
                if i != -1:
                    s = s[:i] + rule[1] + s[i + len(rule[0]):]
                    change = True
                    break
        return s
    
    def critical_pairs(self, rule_1, rule_2):
        s = rule_1[0]
        t = rule_2[0]
        
        pairs = []
        for k in range(1, len(t)):
            if(s.endswith(t[0:k])):
                pairs.append((rule_1[1] + t[k:], s[:-k] + rule_2[1]))
        for k in range(1, len(s)):
            if(t.endswith(s[0:k])):
                pairs.append((rule_2[1] + s[k:], t[:-k] + rule_1[1]))

        return pairs
    
    def compare(self, s, t):
        # Use SHORT(REV?)LEX
        d = len(s) - len(t)
        if d != 0:
            return d
        
        for i in reversed(range(len(s))):
            d = ord(s[i]) - ord(t[i])
            if d != 0:
                return d
            
        return 0
    
    def knuth_bendix(self, height):        
        updates = True
        while updates:
            updates = False
            for i in range(len(self.rules)):
                rule_i = self.rules[i]
                for j in range(i):
                    rule_j = self.rules[j]
                    cpairs = self.critical_pairs(rule_i, rule_j)

                    cpairs = [ (self.apply(p[0]), self.apply(p[1])) for p in cpairs ]
                    cpairs = [ p for p in cpairs if p[0] != p[1] ]
                    cpairs = [ ((p[1], p[0]) if self.compare(p[0], p[1]) < 0 else p) for p in cpairs ]

                    if cpairs:
                        print("Critical pairs from " + str(rule_i) + " and " + str(rule_j) + ": " + str(cpairs))

                    for p in cpairs:
                        if len(p[0]) <= height:                    
                            self.add_rule(p[0], p[1])
                            updates = True
                            
        # Remove obsolete rules
        self.rules = [ rule for rule in self.rules if not self.is_obsolute(rule) ]
        
    def is_obsolute(self, r):
        s = r[0]
        
        updates = True
        while updates:
            updates = False
            for rule in self.rules:
                if rule == r:
                    continue
                i = s.find(rule[0])
                if i != -1:
                    s = s[:i] + rule[1] + s[i + len(rule[0]):]
                    updates = True
                    break

        return s == r[1]

### Tests

In [76]:
rewriter = Rewriter()

rewriter.add_rule("ab", "")
rewriter.add_rule("ba", "")
rewriter.add_rule("xy", "")
rewriter.add_rule("yx", "")
rewriter.add_rule("ax", "xa")

rewriter.knuth_bendix(10)
rewriter.rules

Critical pairs from ('ax', 'xa') and ('ba', ''): [('bxa', 'x')]
Critical pairs from ('ax', 'xa') and ('xy', ''): [('xay', 'a')]
Critical pairs from ('bxa', 'x') and ('ab', ''): [('bx', 'xb')]
Critical pairs from ('xay', 'a') and ('yx', ''): [('ay', 'ya')]
Critical pairs from ('bx', 'xb') and ('xy', ''): [('xby', 'b')]
Critical pairs from ('ay', 'ya') and ('ba', ''): [('bya', 'y')]
Critical pairs from ('xby', 'b') and ('yx', ''): [('by', 'yb')]


[('ab', ''),
 ('ba', ''),
 ('xy', ''),
 ('yx', ''),
 ('ax', 'xa'),
 ('bx', 'xb'),
 ('ay', 'ya'),
 ('by', 'yb')]

In [83]:
rewriter = Rewriter()

rewriter.add_rule("ax", "")
rewriter.add_rule("xa", "")
rewriter.add_rule("by", "")
rewriter.add_rule("yb", "")
rewriter.add_rule("ab", "f")
rewriter.add_rule("ba", "f")

rewriter.knuth_bendix(10)
rewriter.rules

Critical pairs from ('ab', 'f') and ('xa', ''): [('xf', 'b')]
Critical pairs from ('ab', 'f') and ('by', ''): [('fy', 'a')]
Critical pairs from ('ba', 'f') and ('ax', ''): [('fx', 'b')]
Critical pairs from ('ba', 'f') and ('yb', ''): [('yf', 'a')]
Critical pairs from ('ba', 'f') and ('ab', 'f'): [('bf', 'fb'), ('af', 'fa')]
Critical pairs from ('fx', 'b') and ('xf', 'b'): [('bx', 'xb')]
Critical pairs from ('yf', 'a') and ('fy', 'a'): [('ay', 'ya')]
Critical pairs from ('bx', 'xb') and ('yb', ''): [('yxb', 'x')]
Critical pairs from ('ay', 'ya') and ('xa', ''): [('xya', 'y')]
Critical pairs from ('yxb', 'x') and ('by', ''): [('xy', 'yx')]
Critical pairs from ('yxb', 'x') and ('bx', 'xb'): [('yxxb', 'xx')]
Critical pairs from ('yxxb', 'xx') and ('bx', 'xb'): [('yxxxb', 'xxx')]
Critical pairs from ('yxxxb', 'xxx') and ('bx', 'xb'): [('yxxxxb', 'xxxx')]
Critical pairs from ('yxxxxb', 'xxxx') and ('bx', 'xb'): [('yxxxxxb', 'xxxxx')]
Critical pairs from ('yxxxxxb', 'xxxxx') and ('bx', 'xb'):

[('ax', ''),
 ('xa', ''),
 ('by', ''),
 ('yb', ''),
 ('ab', 'f'),
 ('ba', 'f'),
 ('xf', 'b'),
 ('fy', 'a'),
 ('fx', 'b'),
 ('yf', 'a'),
 ('bf', 'fb'),
 ('af', 'fa'),
 ('bx', 'xb'),
 ('ay', 'ya'),
 ('yxb', 'x'),
 ('xy', 'yx'),
 ('yxxb', 'xx'),
 ('yxxxb', 'xxx'),
 ('yxxxxb', 'xxxx'),
 ('yxxxxxb', 'xxxxx'),
 ('yxxxxxxb', 'xxxxxx'),
 ('yxxxxxxxb', 'xxxxxxx'),
 ('yxxxxxxxxb', 'xxxxxxxx')]

In [74]:
rewriter = Rewriter()

rewriter.add_rule("fg", "")
rewriter.add_rule("gf", "h")

rewriter.knuth_bendix(10)
rewriter.rules

Critical pairs from ('gf', 'h') and ('fg', ''): [('hg', 'g'), ('fh', 'f')]
Critical pairs from ('hg', 'g') and ('gf', 'h'): [('hh', 'h')]


[('fg', ''), ('gf', 'h'), ('hg', 'g'), ('fh', 'f'), ('hh', 'h')]